In [1]:
import pandas as pd
import json
import re

balance_sheet_path    = "C:\\Users\\paulcassidy\\OneDrive - Archdiocese of Chicago\\Documents\\JSON_Parsing\\RawData\\BalanceSheet.json"
transaction_list_path = "C:\\Users\\paulcassidy\\OneDrive - Archdiocese of Chicago\\Documents\\JSON_Parsing\\RawData\\TransactionList.json"
general_ledger_path   = "C:\\Users\\paulcassidy\\OneDrive - Archdiocese of Chicago\\Documents\\JSON_Parsing\\RawData\\GeneralLedger.json"

with open(balance_sheet_path) as f:
    balance_sheet = json.load(f)

with open(transaction_list_path) as f:
    transaction_list = json.load(f)

with open(general_ledger_path) as f:
    general_ledger = json.load(f)

In [13]:
def parse_record_gl(data):
    """
    Iterate through ColData or Header.ColData data frame. 
    Check what kind of data each data frame contains.
    Parse each record in data frame accordingly and return values.
    """

    # regular expression matching date format
    rex = re.compile("^[0-9]{4}-[0-9]{2}-[0-9]{2}$")

    # check which ColData data frame is being parsed
    for row in range(data.shape[0]):
        # transaction detail data frame
        if re.search(rex, data['value'][0]):
            td   = data['value'][0]
            tt   = data['value'][1]
            ttid = data['id'][1]
            dn   = data['value'][2]
            vn   = data['value'][3]
            vid  = data['id'][3]
            m    = data['value'][4]
            s    = data['value'][5]
            sid  = data['id'][5]
            ta   = data['value'][6]
            nab  = data['value'][7]
            
            vals = {
                'TransactionDate' : td,
                'TransactionType' : tt,
                'TransactionTypeID' : ttid,
                'DocumentNumber' : dn,
                'VendorName' : vn,
                'VendorID' : vid,
                'Memo' : m,
                'Split' : s,
                'SplitID' : sid,
                'TransactionAmount' : ta,
                'NewAccountBalance' : nab
                }

            # replace missing values           
            for k in vals:
                if not vals[k]:
                    vals[k] = "NA"
            
            return vals

        # beginning balance data frame
        elif data['value'][0] == "Beginning Balance":
            bb   = data['value'][7]
            return {
                'BeginningBalance' : bb
            }

        # name data frame
        else:
            if 'id' in data:
                an  = data['value'][0]
                aid = data['id'][0]
            else:
                an  = data['value'][0]
                aid = "NA"
                
            return {
                'AccountName' : an,
                'AccountID' : aid
            } 

def json_crawler_gl(data):
    # If data is dictionary returned by json.load(), then flatten it to initial pandas data frame
    if isinstance(data, dict):
        data = pd.json_normalize(data)
        return_value = json_crawler_gl(data)
        return return_value 
    
    # Crawl through JSON file, normalizing by different keys to find account data and save to lst
    else:
        # Initialize list to hold account data objects
        lst = []
        acct_names = []
        
        for row in range(data.shape[0]):
            # Normalize by 'ColData'
            try:
                record_data = pd.json_normalize(data['ColData'][row])
                lst.append(parse_record_gl(record_data))
            except:
                pass
            # Normalize by 'Header.ColData'
            try:
                record_data = pd.json_normalize(data['Header.ColData'][row])
                lst.append(parse_record_gl(record_data))
                # acct_names.append(
                #     {'AccountName' : parse_record_gl_tl(record_data)['AccountName']},
                #     {'AccountID' : parse_record_gl_tl(record_data)['AccountID']})
            except:
                pass
            # Normalize by 'Rows.Row'
            try:
                row_data = pd.json_normalize(data['Rows.Row'][row])
                # Capture lst and return value before recursive function call
                return_value = json_crawler_gl(row_data)
                lst = lst + return_value
            except:
                pass

        return lst

def list_df_csv(lst, prefix, filename):
    df = pd.DataFrame(lst)
    df.to_csv(f"{prefix}/{filename}.csv", index = False)


In [14]:
list_df_csv(json_crawler_gl(general_ledger), "C:\\Users\\paulcassidy\\OneDrive - Archdiocese of Chicago\\Documents\\JSON_Parsing\\Output", "test_gl")

## Need to handle different column names for the transaction list report

In [10]:
def parse_record_tl(data):
    """
    Iterate through ColData or Header.ColData data frame. 
    Check what kind of data each data frame contains.
    Parse each record in data frame accordingly and return values.
    """

    # regular expression matching date format
    rex = re.compile("^[0-9]{4}-[0-9]{2}-[0-9]{2}$")

    # check which ColData data frame is being parsed
    for row in range(data.shape[0]):
        # transaction detail data frame
        if re.search(rex, data['value'][0]):
            td   = data['value'][0]
            tt   = data['value'][1]
            ttid = data['id'][1]
            dn   = data['value'][2]
            p    = data['value'][3]
            vn   = data['value'][4]
            vid  = data['id'][4]
            m    = data['value'][5]
            s    = data['value'][6]
            sid  = data['id'][6]
            ta   = data['value'][7]
            
            vals = {
                'TransactionDate' : td,
                'TransactionType' : tt,
                'TransactionTypeID' : ttid,
                'DocumentNumber' : dn,
                'Posting' : p,
                'VendorName' : vn,
                'VendorID' : vid,
                'Memo' : m,
                'Split' : s,
                'SplitID' : sid,
                'TransactionAmount' : ta
                }

            # replace missing values           
            for k in vals:
                if not vals[k]:
                    vals[k] = "NA"
            
            return vals

def json_crawler_tl(data):
    # If data is dictionary returned by json.load(), then flatten it to initial pandas data frame
    if isinstance(data, dict):
        data = pd.json_normalize(data)
        return_value = json_crawler_tl(data)
        return return_value 
    
    # Crawl through JSON file, normalizing by different keys to find account data and save to lst
    else:
        # Initialize list to hold account data objects
        lst = []
        acct_names = []
        
        for row in range(data.shape[0]):
            # Normalize by 'ColData'
            try:
                record_data = pd.json_normalize(data['ColData'][row])
                lst.append(parse_record_tl(record_data))
            except:
                pass
            # Normalize by 'Header.ColData'
            try:
                record_data = pd.json_normalize(data['Header.ColData'][row])
                lst.append(parse_record_tl(record_data))
                # acct_names.append(
                #     {'AccountName' : parse_record_gl_tl(record_data)['AccountName']},
                #     {'AccountID' : parse_record_gl_tl(record_data)['AccountID']})
            except:
                pass
            # Normalize by 'Rows.Row'
            try:
                row_data = pd.json_normalize(data['Rows.Row'][row])
                # Capture lst and return value before recursive function call
                return_value = json_crawler_tl(row_data)
                lst = lst + return_value
            except:
                pass

        return lst

def list_df_csv(lst, prefix, filename):
    df = pd.DataFrame(lst)
    df.to_csv(f"{prefix}/{filename}.csv", index = False)


In [11]:
list_df_csv(json_crawler_tl(transaction_list), "C:\\Users\\paulcassidy\\OneDrive - Archdiocese of Chicago\\Documents\\JSON_Parsing\\Output", "test_tl")